In [ ]:
import numpy as np
import matplotlib.pyplot as plt
np.set_printoptions(precision=3)

import ksource_py as ks

In [ ]:
%matplotlib notebook

# Abrir tally
#### Se recomienda completar plantilla en una celda duplicada

In [ ]:
outputfile = "filename"
tallyname = "tally"
spectrum = "spectrum"

tally = ks.T4Tally(outputfile, tallyname)

In [ ]:
outputfile = "2_bunker_n/2_bunker_n.out"
tallyname = "mapa_dosis_ne"
spectrum = "Decay/Fe59.csv"

tally = ks.T4Tally(outputfile, tallyname)

# Crear graficos

In [ ]:
cell = None # Indice de celda en direccion normal a pantalla. None: promediar todas

[fig, [scores,errs]] = tally.plot2D(['z', 'x'], cell=cell)

# Guardar como lista de tracks

In [ ]:
tracksname = folder+"/activ_fe.ssv"

tally.save_tracks(tracksname)

# Crear KSource

In [ ]:
# Abrir lista de particulas

tracks = ["Decay/Fe59.csv", "2_bunker_n_bw0/activ_fe.ssv", None]
readformat = ["Decay", "SSVtally", "Isotrop"]
trasl = None
rot = None
x2z = False

set_params = True
plist = ks.PList(readformat, tracks, trasl=trasl, rot=rot, switch_x2z=x2z, set_params=set_params)

# Crear Metrica

trasl = None
rot = None
metric = ks.Geometry([ks.Lethargy(10), ks.Vol(), ks.Isotrop()], trasl=trasl, rot=rot)

#Crear KSource

J = 1
s = ks.KSource(plist, metric, bw="silv", J=J)

In [ ]:
# Ajustar KSource

N = 1e5 # Cantidad de particulas a utilizar
s.fit(N)
s.bw /= 5 # Para graficos integrales puede usarse una bw mas baja

# Evaluar estadistica

In [ ]:
Emin = 1e-9
dzmax = np.sqrt(1-0.01**2)
box = ks.Box([Emin,None,None,None,None,None,dzmax], None)

parts,ws = plist.get()
tracks = ks.Tracks(parts, ws, box)

N,mn,err = tracks.intensity(steps=100)
plt.show()

N,mn,err = tracks.mean(var=3, steps=100)
plt.show()

# Optimizar BW (revisar)

In [ ]:
# Metodo de Silverman

s.bw_method = "silv"
N = 1E4
s.fit(N, N_tot=2991283)
s.save_bw(tracksfile[:-4]+"_bw_silv.txt")

In [ ]:
# Metodo de Max Likelihood Cross Validation

s.bw_method = "mlcv"
N = 1E4
s.fit(N, N_tot=s.plist.N, steps=20, max_fact=2, shift=.5)
s.save_bw(tracksfile[:-4]+"_bw_mlcv.txt")

In [ ]:
# Metodo de K Nearest Neighbours

s.bw_method = "knn"
N = 1e5
s.fit(N)#, N_tot=100000)#2991283)
s.save_bw(tracksfile[:-4]+"_bw_knn.txt")

print("Mean bw:")
print(s.bw.mean(axis=0))

In [ ]:
# Graficos de bw variable

if N>s.plist.N: N = s.plist.N
bws = (s.bw / s.metric.std(vecs=s.vecs))[:,0]

Es = s.metric.E.inverse_transform(s.vecs[:N,0])
plt.plot(Es, bws, 'o')
plt.ylabel("bandwidth")
plt.xlabel("E [MeV]")
plt.xscale("log")
plt.show()

xs = s.vecs[:N,1]
ys = s.vecs[:N,2]
plt.scatter(xs, ys, c=bws)
plt.colorbar()
plt.xlabel("x [cm]")
plt.ylabel("y [cm]")
plt.show()

mus = s.vecs[:N,5]
plt.plot(mus, bws, 'o')
plt.ylabel("bandwidth")
plt.xlabel("mu []")
plt.show()

# Guardar KSource

In [ ]:
filename = tracksfile[:-4]+"_source.txt"
bwfilename = tracksfile[:-4]+"_bw_knn.txt"
s.save(filename, bwfilename)